In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [50]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

## 데이터 수집

In [51]:
train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

# from sklearn.datasets import load_diabetes
# data_diabetes = load_diabetes()
# features, label = data_diabetes.data, data_diabetes.target
# features.shape, label.shape

## 데이터 전처리
- 데이터 분석

In [52]:
# 1. features(입력 데이터)와 label(정답) 타겟 변수 분리
features = train_data.drop(columns=["SalePrice"])  
label = train_data["SalePrice"]  
test_features = test_data  

# 2. 연속형 변수 선택(숫자형 데이터만 선택)
numerical_features = features.select_dtypes(include=["int64","float64"]).columns

# 3. StandardScaler 적용 (학습 데이터 기준으로 스케일링)
scaler = StandardScaler()

# 학습 데이터 변환
features_scaled = scaler.fit_transform(features[numerical_features])  
features_scaled = pd.DataFrame(features_scaled, columns=numerical_features)

# 테스트 데이터 변환 (train 기준으로 변환해야 함!)
test_features_scaled = scaler.transform(test_features[numerical_features])  
test_features_scaled = pd.DataFrame(test_features_scaled, columns=numerical_features)

## 데이터 분할
- train, test, validation

In [53]:
from sklearn.model_selection import train_test_split
# train_features, test_features, train_label, test_label = train_test_split(features, label, test_size=0.2, random_state=42)

In [54]:
import torch
import numpy as np

- 텐서변화

In [58]:
# NaN(결측값)이 있다면 처리하기
train_features = train_features.fillna(0)
test_features = test_features.fillna(0)

In [59]:
# train_features에서 숫자형 데이터만 선택
train_features_numeric = train_features.select_dtypes(include=["int64", "float64"])
test_features_numeric = test_features.select_dtypes(include=["int64", "float64"])

# 텐서 변환
train_features_tensor = torch.tensor(train_features_numeric.values, dtype=torch.float32)
test_features_tensor = torch.tensor(test_features_numeric.values, dtype=torch.float32)

## 모델 학습

In [60]:
# Neural Network model linear regression
class LinearRegressionNNM(torch.nn.Module):
    def __init__(self, input_dim):    # input_dim:features 열 갯수
        super(LinearRegressionNNM, self).__init__()
        # super(self).__init__()
        self.hidden_1 = torch.nn.Linear(input_dim, 64)
        self.hidden_2 = torch.nn.Linear(64, 32)
        self.hidden_3 = torch.nn.Linear(32, 16)
        self.output = torch.nn.Linear(16, 1)
        self.relu = torch.nn.ReLU()  # Activation function

    def forward(self, x):
        x = self.relu(self.hidden_1(x))  # hidden layer 1
        x = self.relu(self.hidden_2(x))  # hidden layer 2
        x = self.relu(self.hidden_3(x))  # hidden layer 3
        out = self.output(x)   # result
        return out


In [62]:
# train_features_tensor.shape, train_label_tensor.shape

In [71]:
model = LinearRegressionNNM(1)

In [72]:
model

LinearRegressionNNM(
  (hidden_1): Linear(in_features=1, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [73]:
criterion = torch.nn.MSELoss()  # Loss function
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # optimizer function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # optimizer function

In [74]:
# 반복 학습
# for epoch in range(10):
# # for epoch in range(1000):
# # for epoch in range(10000):
#     # Forward pass: Compute predicted y by passing x to the model
#     pred_y = model(train_features_tensor)

#     # Compute and print loss
#     loss = criterion(pred_y, train_label_tensor)

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     if epoch % 1000 == 0:
#       print('epoch: {}, loss: {}'.format(epoch, loss.item()))

for epoch in range(100):
    predictions = model(train_features_tensor)
    loss = criterion(predictions, train_label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/100], Loss: {loss.item():.4f}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1168x37 and 1x64)

## 모델 평가

In [ ]:
model.eval()

In [ ]:
with torch.no_grad():   # 학습 목적 아닌 평가 목적 위해 고정
    pred_y = model(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor)  # 예측도 같은 Loss function
    print('loss: {}'.format(loss.item()))

In [ ]:
# 결과 값(예측값, 원래값)
pred_y[0], train_label[0]

## 모델 배포